In [2]:
import numpy as np 
import scipy.io as sio
import glob 



In [3]:
driving_cycle_paths = glob.glob("../data/driving_cycles/all/*.mat")

In [4]:
def get_min_max(driving_cycle_paths):
    v_max = 0 
    v_diff_min = 0 
    v_diff_max = 0 
    
    for cycle_path in driving_cycle_paths: 
        drv_cycle = sio.loadmat(cycle_path)
        v_veh = drv_cycle["sch_cycle"][:, 1]
        v_diff_veh = np.diff(v_veh)
        
        for v in v_veh: 
            if v > v_max: 
                v_max = v 
        
        for v_diff in v_diff_veh: 
            if v_diff > v_diff_max: 
                v_diff_max = v_diff 
            if v_diff < v_diff_min: 
                v_diff_min = v_diff 
                
    return v_max, v_diff_min, v_diff_max 

In [9]:
v_max, v_diff_min, v_diff_max = get_min_max(driving_cycle_paths)

v_diff_min = - v_diff_max 

print(v_max, v_diff_min, v_diff_max)



33.349183999999994 -4.088888888888889 4.088888888888889


In [10]:
v_grid_num = 20  
a_grid_num = 20 

v_grid = np.linspace(0, v_max, v_grid_num + 1) 
a_grid = np.linspace(v_diff_min, v_diff_max, a_grid_num + 1)



print(a_grid)

[-4.08888889 -3.68       -3.27111111 -2.86222222 -2.45333333 -2.04444444
 -1.63555556 -1.22666667 -0.81777778 -0.40888889  0.          0.40888889
  0.81777778  1.22666667  1.63555556  2.04444444  2.45333333  2.86222222
  3.27111111  3.68        4.08888889]


In [11]:
def get_indices(v, a, v_grid, a_grid):
    for i in range(len(v_grid) - 1): 
        if v_grid[i] <= v < v_grid[i+1]: 
            break 
    
    for j in range(len(a_grid) - 1): 
        if a_grid[j] <= a < a_grid[j+1]: 
            break
            
    return i, j 
            
    

In [13]:
def get_transition_matrix(driving_cycle_paths, v_grid, a_grid): 
    transition_matrix = np.zeros((v_grid_num, a_grid_num))
    
    for cycle_path in driving_cycle_paths: 
        drv_cycle = sio.loadmat(cycle_path)
        v_veh = drv_cycle["sch_cycle"][:, 1]
        
        for t in range(len(v_veh)-1):
            v = v_veh[t]
            a = v_veh[t+1] - v_veh[t]
            
            i, j = get_indices(v, a, v_grid, a_grid)
            transition_matrix[i, j] += 1 
            
    return transition_matrix 
            

transition_matrix = get_transition_matrix(driving_cycle_paths, v_grid, a_grid)

In [20]:
transition_prob_matrix = np.zeros_like(transition_matrix)

for i in range(len(transition_matrix)): 
    state_sum = np.sum(transition_matrix[i]) 
    state_prob = transition_matrix[i] / state_sum 
    
    transition_prob_matrix[i] = state_prob 
#     print(sum(state_prob))
    
# print(transition_prob_matrix)

# transition matrix로 speed profile 만들기 